In [34]:
import pyvista as pv
# import topogenesis  as tp
import numpy as np
from honeybee_plus.hbsurface import HBSurface
from honeybee_plus.radiance.recipe.solaraccess.gridbased import SolarAccessGridBased
from honeybee_plus.radiance.analysisgrid import AnalysisGrid
import os
import ladybug as lb

cwd = os.getcwd()
src = os.path.dirname(cwd)

In [35]:
filepath = os.path.join(src, "data", "test_room_02.obj")
mesh = pv.read(filepath)
# print(mesh.is_all_triangles())

# srf_dict = {
#     'name': 'floor', 
#     'vertices': [[(0, 0, 3.2), (-4.242640687119285, 4.242640687119286, 3.2), (-1.2727922061357848, 7.212489168102785, 3.2), (2.9698484809835, 2.9698484809834995, 3.2)]], 
#     'surface_material': {
#         'modifier': 'void', 
#         'type': 'plastic', 
#         'name': 'generic_floor', 
#         'r_reflectance': 0.2, 
#         'g_reflectance': 0.2, 
#         'b_reflectance': 0.2, 
#         'specularity': 0.0, 
#         'roughness': 0.0}, 
#     'surface_type': 2.0
# }
face_list = list(mesh.faces)
hb_surfaces = []
e = 0
while e < len(face_list):
    v_count = face_list[e]
    vertices = []
    for v in range(v_count):
        e += 1
        vertices.append(list(mesh.points[face_list[e]]))
    srf_dict = {
        "name": "",
        "vertices": vertices, # [[(x, y, z), (x1, y1, z1), (x2, y2, z2)]],
        "surface_material": {
            'modifier': 'void', 
            'type': 'plastic', 
            'name': 'generic_floor', 
            'r_reflectance': 0.2, 
            'g_reflectance': 0.2, 
            'b_reflectance': 0.2, 
            'specularity': 0.0, 
            'roughness': 0.0},
        "surface_type": 0  # 0: wall, 5: window
        }
    hbsrf = HBSurface.from_json(srf_dict)
    hb_surfaces.append(hbsrf)
    e += 1


# print(hb_surfaces)
# # print(type(mesh))
# # print(mesh.points)
# print(mesh.faces)

In [56]:
# # generate grid of test points
# analysis_grid = room.generate_test_points(grid_size=0.5, height=0.75)
test_pts = [(1, 1, 1),(6, 1, 1),(1, 6, 1),(1, 1, 6)]
test_vec = [(0, 0, 1),(0, 0, 1),(0, 0, 1),(0, 0, 1)]
analysis_grid = AnalysisGrid.from_points_and_vectors(test_pts, test_vec, 'test_grid')

# initiate sunpath
# NOTE: I had to insert the lon and lat inversely to get it to work properly
sp = lb.sunpath.Sunpath(longitude=4.3571, latitude=52.0116)

# define sun hours : A list of hours of the year for each sun vector
# there are 8760 hours in a year, so the following integers refer to specific hours throughout the year
# hoys = [1908, 2000, 2530, 3254, 3658, 4000, 4116, 6324, 8508]
# hoys = list(range(8760))
# # compute the sun from hoys
# suns = [sp.calculate_sun_from_hoy(hoy) for hoy in hoys]
# # extract sun vectors from suns
# sun_vectors = [sun.sun_vector.to_array() for sun in suns]
hoys = []
sun_vectors = []
for i in range(8760):
    # compute the sun object
    sun = sp.calculate_sun_from_hoy(i)
    # extract the sun vector
    sun_vector = sun.sun_vector.to_array()
    # apparantly, if the Z component of sun vector is positive, it is under the horizon 
    if sun_vector[2] < 0.0:
        hoys.append(i)
        sun_vectors.append(sun_vector)

# # define sun vectors
# sun_vectors = [
#             (-0.810513, 0.579652, -0.084093), (-0.67166, 0.702357, -0.235729),
#             (-0.487065, 0.798284, -0.354275), (-0.269301, 0.8609, -0.431657),
#             (-0.033196, 0.885943, -0.462605), (0.20517, 0.871705, -0.445013),
#             (0.429563, 0.819156, -0.380077), (0.624703, 0.731875, -0.272221),
#             (0.777301, 0.615806, -0.128788)]


# # put the recipe together
# rp = GridBased(sky=sky, analysis_grids=(analysis_grid,), simulation_type=0, hb_objects=(room,))
rp = SolarAccessGridBased(sun_vectors, hoys, [analysis_grid], hb_objects=hb_surfaces)

Found 78 opaque surfaces.
Found 0 fenestration surfaces.
Found 0 window-groups.


In [57]:
# write simulation to folder
batch_file = rp.write(
    target_folder='.', project_name='external_room_sun')

Writing recipe contents to: ./external_room_sun/solaraccess


In [58]:
# run the simulation
for com in rp._commands:
    print(com)
rp.run(batch_file, debug=False)

# https://www.radiance-online.org/learning/documentation/manual-pages/pdfs/rmtxop.pdf
# Eleonora: rmtxop multiplies the daylight coefficient matrices with the sky vectors. This is necessary for getting illuminance from daylight coefficients computed w.r.t. rgb components. 
# Sun Access of points, and how to interpret them https://floyd.lbl.gov/radiance/refer/usman2.pdf

#!/usr/bin/env bash



cd ./external_room_sun/solaraccess

/usr/local/radiance/bin/oconv -f scene/opaque/external_room_sun..opq.mat scene/opaque/external_room_sun..opq.rad scene/glazing/external_room_sun..glz.mat scene/glazing/external_room_sun..glz.rad sky/analemma.rad > external_room_sun.oct
/usr/local/radiance/bin/rcontrib -aa 0.0 -ab 0 -ad 512 -ar 16 -as 128 -dc 1.0 -dj 0.0 -dp 64 -ds 0.5 -dr 0 -dt 0.0 -I -lr 4 -lw 0.05 -M ./sky/analemma.mod -ss 0.0 -st 0.85 -y 4 external_room_sun.oct < external_room_sun.pts > result/external_room_sun.dc
/usr/local/radiance/bin/rmtxop -c 47.4 119.9 11.6 -fa result/external_room_sun.dc > result/external_room_sun.ill


True

In [59]:
# results - in this case it will be an analysis grid
result = rp.results()[0]
print("Daylight autonomy, Continious daylight autonomy, Useful daylight illuminance, Less than UDI, More than UDI")
print(result.annual_metrics())
result_points = [[p[0], p[1], p[2]] for p in result.points]

Unloading the current values from the analysis grids.
Daylight autonomy, Continious daylight autonomy, Useful daylight illuminance, Less than UDI, More than UDI
([0.0, 0.0, 0.0, 0.0], [0.0, 0.24430281962146652, 0.11123986095017559, 0.3333333333333383], [0.0, 0.0, 0.0, 0.0], [100.0, 100.0, 100.0, 100.0], [0.0, 0.0, 0.0, 0.0])


In [60]:
# Plot using the ITKplotter
pl = pv.PlotterITK()
# add the external mesh
pl.add_mesh(mesh, smooth_shading=True)
# add points of interest : blue
pl.add_points(pv.PolyData(result_points), color='#2499ff')
# add sun points : orange
pl.add_points(pv.PolyData(np.array(sun_vectors) * -20), color='#ffa500')
pl.show(True)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…